In [1]:
import os 
import pandas as pd 
from string import punctuation
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from langdetect import detect, DetectorFactory, detect_langs

stop_words = set(stopwords.words('english'))

DIRECTORY = 'Data/Tables'
Text = []
Labels = []
file_names=[]

ch = ['����', 'NULL', 'Null', 'null', '&NBSP', '&Nbsp', '&nbsp', '&nbsp;', '????', '�']

# Retrieve labels :
labels_dic = {}
df_labels = pd.read_csv('Data/classes_complete.csv', header=None)

print(len(df_labels))


1688


In [2]:
#Set Seed
import random
import numpy as np
import torch
random.seed(555)
np.random.seed(555)
torch.manual_seed(555)

In [3]:
DetectorFactory.seed = 42
for i in range(len(df_labels)):
	labels_dic[df_labels[0][i].split('.tar.gz')[0]] = df_labels[1][i]

for filename in os.listdir(DIRECTORY):
	f = os.path.join(DIRECTORY, filename)
	# checking if it is a file
	if os.path.isfile(f):
		with open(f, encoding='utf-8') as ff:
			lines = []
			for line in ff:
				#print(line)
				line_new = line.replace('"', '').replace("'", '').replace('_', '').split(',')
				lines.append(line_new)

			# Check columns size
			columns_check = 0
			for column in lines[0]:
				if len(column) > 1:
					columns_check = columns_check + 1
			if columns_check == len(lines[0]):				
				# Check language : keep only corpora in english language
				text_columns = ' '.join(lines[0])
				try:
					lang = detect(text_columns)
					
				except :
					pass
				else:
					if lang == 'en':
						#print(text)
						text_df =  ''
						for line in lines:
							# Join string
							txt_line = ' '.join(line)
							
							# remove bad characters
							txt_line = txt_line.replace('����', '')
							txt_line = txt_line.replace('NULL', '')
							txt_line = txt_line.replace('Null', '')
							txt_line = txt_line.replace('null', '')
							txt_line = txt_line.replace('NBSP', '')
							txt_line = txt_line.replace('�', '')
							txt_line = txt_line.replace('Nbsp', '')
							txt_line = txt_line.replace('nbsp', '')

							# Remove punctuation from text
							punctuation = punctuation.replace('_', '')
							punctuation = punctuation.replace('_ _', '')
							punctuation = punctuation.replace(' _', '')
							punctuation = punctuation.replace('_ ', '')                            

							for x in punctuation:
								txt_line = txt_line.replace(x, '')

							# Remove 'return to line character'
							txt_line = txt_line.replace('\n', '')

							# Build text(str) for df
							text_df = text_df.strip() + '@' + txt_line

						lab = filename.split('.csv')[0]
						if lab in labels_dic.keys():
							label = labels_dic[lab]
							Labels.append(label)
							file_names.append(lab)
							Text.append(text_df)

In [4]:
print(len(df_labels))
print(len(Text))
print(len(Labels))
print(len(file_names))
# Save
with open("TextPre.csv", 'w', encoding='utf-8') as f:
	f.write('Text' + ',' + 'Labels' + ','+ 'file_names\n' )
	for i in range(len(Text)):
		f.write(str(Text[i]) + ',' + str(Labels[i])+ ',' + str(file_names[i]) + '\n')

1688
927
927
927


In [5]:
clean_df = pd.read_csv("TextPre.csv")
# Stop-word removal
clean_df['Text'] = clean_df['Text'].apply(lambda x : [w for w in x.split() if not w.lower() in stop_words]).apply(lambda x : ' '.join(x))
lemmatizer = WordNetLemmatizer()

clean_df['Text'] = clean_df['Text'].apply(lambda x : [lemmatizer.lemmatize(w) for w in x.split()]).apply(lambda x : ' '.join(x))

In [6]:
df=clean_df
df

,Text,Labels,file_names
0,@Title Connecticut Office Cleaning@URL httpwww...,Website,10057926_0_6000962521981596991
1,@product length 180 inches@product width 039 i...,Thing,10068410_0_1159206472287350411
2,@RANK COMPANY 3 YEAR GROWTH REVENUE million ST...,Company,10133296_0_1835025541686843841
3,@Name 5Year Total Return Sales bil Net Income ...,Company,10141841_0_473063208079217959
4,@title author source@adventures huckleberry fi...,Book,10151359_0_8168779773862259178
...,...,...,...
922,@ charles danny duncan tom@mean 753 708 693 71...,Thing,99022135_0_7554945610165642414
923,@no name foot met grid ref area sum trips@1 sc...,Mountain,99070098_0_2074872741302696997
924,@Country Currency Abbreviation@Argentina peso ...,Country,9931816_0_2647599887020622155
925,@ add correction comment review thought buildi...,Thing,99542663_0_7603724551662549602


In [7]:
count_df = clean_df.Labels.value_counts()
count_df

Thing                  431
Country                125
Company                 77
Film                    23
Bird                    21
                      ... 
ProgrammingLanguage      1
Church                   1
Wrestler                 1
Dog                      1
AcademicJournal          1
Name: Labels, Length: 68, dtype: int64

In [8]:
count_df = count_df.to_frame().reset_index().rename(columns={'index':'Labels', 'Labels':'Count'})

In [9]:
count_df

,Labels,Count
0,Thing,431
1,Country,125
2,Company,77
3,Film,23
4,Bird,21
...,...,...
63,ProgrammingLanguage,1
64,Church,1
65,Wrestler,1
66,Dog,1


In [10]:
count_df=count_df[count_df['Count'] >= 4]

In [11]:
count_df = count_df[count_df['Labels'] != 'Thing']
count_df

,Labels,Count
1,Country,125
2,Company,77
3,Film,23
4,Bird,21
5,VideoGame,16
6,PoliticalParty,13
7,Airport,13
8,City,12
9,Plant,12
10,University,11


In [12]:
count_df = count_df.drop_duplicates('Labels')

In [13]:
count_df = pd.merge(df, count_df, on='Labels', how='inner')

In [14]:
count_df

,Text,Labels,file_names,Count
0,@RANK COMPANY 3 YEAR GROWTH REVENUE million ST...,Company,10133296_0_1835025541686843841,77
1,@Name 5Year Total Return Sales bil Net Income ...,Company,10141841_0_473063208079217959,77
2,@Name 5Year Total Return Sales bil Net Income ...,Company,10786782_0_7941448888047609465,77
3,@Company Symbol Industry Date Added@3M MMM Div...,Company,12271141_0_213693783755148562,77
4,@company symbol industry date added@3m mmm div...,Company,12271141_0_8517913935669973086,77
...,...,...,...,...
424,@network affiliate local channel ird channel b...,TelevisionStation,38146710_0_3777366193503047044,6
425,@Network Affiliate Local Channel Dish Channel@...,TelevisionStation,38238783_0_4144778912351744096,6
426,@City Country Station Genre Stream Language@An...,TelevisionStation,49688745_0_5865173677570896512,6
427,@City Country Station Genre Stream Language@Sy...,TelevisionStation,79058714_0_9111059240060469828,6


In [15]:
count_df.drop("Count", axis=1, inplace=True)
count_df

,Text,Labels,file_names
0,@RANK COMPANY 3 YEAR GROWTH REVENUE million ST...,Company,10133296_0_1835025541686843841
1,@Name 5Year Total Return Sales bil Net Income ...,Company,10141841_0_473063208079217959
2,@Name 5Year Total Return Sales bil Net Income ...,Company,10786782_0_7941448888047609465
3,@Company Symbol Industry Date Added@3M MMM Div...,Company,12271141_0_213693783755148562
4,@company symbol industry date added@3m mmm div...,Company,12271141_0_8517913935669973086
...,...,...,...
424,@network affiliate local channel ird channel b...,TelevisionStation,38146710_0_3777366193503047044
425,@Network Affiliate Local Channel Dish Channel@...,TelevisionStation,38238783_0_4144778912351744096
426,@City Country Station Genre Stream Language@An...,TelevisionStation,49688745_0_5865173677570896512
427,@City Country Station Genre Stream Language@Sy...,TelevisionStation,79058714_0_9111059240060469828


In [16]:
count_df['Labels'].replace(' ', np.nan, inplace=True)
count_df.dropna(subset=['Labels'], inplace=True)

In [17]:
count_df['Labels'].nunique()

26

In [18]:
count_df = count_df.replace('_', '')
count_df

,Text,Labels,file_names
0,@RANK COMPANY 3 YEAR GROWTH REVENUE million ST...,Company,10133296_0_1835025541686843841
1,@Name 5Year Total Return Sales bil Net Income ...,Company,10141841_0_473063208079217959
2,@Name 5Year Total Return Sales bil Net Income ...,Company,10786782_0_7941448888047609465
3,@Company Symbol Industry Date Added@3M MMM Div...,Company,12271141_0_213693783755148562
4,@company symbol industry date added@3m mmm div...,Company,12271141_0_8517913935669973086
...,...,...,...
424,@network affiliate local channel ird channel b...,TelevisionStation,38146710_0_3777366193503047044
425,@Network Affiliate Local Channel Dish Channel@...,TelevisionStation,38238783_0_4144778912351744096
426,@City Country Station Genre Stream Language@An...,TelevisionStation,49688745_0_5865173677570896512
427,@City Country Station Genre Stream Language@Sy...,TelevisionStation,79058714_0_9111059240060469828


In [19]:
# For attributes and values
count_df.to_csv("TextPre.csv", index=False)

In [20]:
count_df

,Text,Labels,file_names
0,@RANK COMPANY 3 YEAR GROWTH REVENUE million ST...,Company,10133296_0_1835025541686843841
1,@Name 5Year Total Return Sales bil Net Income ...,Company,10141841_0_473063208079217959
2,@Name 5Year Total Return Sales bil Net Income ...,Company,10786782_0_7941448888047609465
3,@Company Symbol Industry Date Added@3M MMM Div...,Company,12271141_0_213693783755148562
4,@company symbol industry date added@3m mmm div...,Company,12271141_0_8517913935669973086
...,...,...,...
424,@network affiliate local channel ird channel b...,TelevisionStation,38146710_0_3777366193503047044
425,@Network Affiliate Local Channel Dish Channel@...,TelevisionStation,38238783_0_4144778912351744096
426,@City Country Station Genre Stream Language@An...,TelevisionStation,49688745_0_5865173677570896512
427,@City Country Station Genre Stream Language@Sy...,TelevisionStation,79058714_0_9111059240060469828


In [21]:
count_df["Text"]=count_df["Text"].str.split(r'\s*@\s*|\s*\@\s*').str[1]

In [22]:
count_df["Text"].replace('\d+', '', regex=True, inplace=True)

In [23]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
count_df['Target'] = le.fit_transform(count_df.Labels)
count_df.columns = ['Text', 'Target', 'file name','Labels']
count_df

,Text,Target,file name,Labels
0,RANK COMPANY YEAR GROWTH REVENUE million STATE,Company,10133296_0_1835025541686843841,6
1,Name Year Total Return Sales bil Net Income mi...,Company,10141841_0_473063208079217959,6
2,Name Year Total Return Sales bil Net Income mi...,Company,10786782_0_7941448888047609465,6
3,Company Symbol Industry Date Added,Company,12271141_0_213693783755148562,6
4,company symbol industry date added,Company,12271141_0_8517913935669973086,6
...,...,...,...,...
424,network affiliate local channel ird channel br...,TelevisionStation,38146710_0_3777366193503047044,23
425,Network Affiliate Local Channel Dish Channel,TelevisionStation,38238783_0_4144778912351744096,23
426,City Country Station Genre Stream Language,TelevisionStation,49688745_0_5865173677570896512,23
427,City Country Station Genre Stream Language,TelevisionStation,79058714_0_9111059240060469828,23


In [24]:
# For attributes only
count_df.to_csv("TextPre1.csv", index=False)